In [1]:
# from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [3]:
file1 = open('../../../CAECseq/CAECseq_viral/alignedReads.sam', 'r')
Lines = file1.readlines()


FileNotFoundError: [Errno 2] No such file or directory: '../../../CAECseq/CAECseq_viral/alignedReads.sam'

In [9]:
count = 0
# Strips the newline character
read_lengths = []
counter = 0
sumOfItAll = 0
print(len(Lines))
for line in Lines:
    print(len(line[33:line.find('	',33)]))
#     print(line[33:(line.find('	',33)+1)])
    words = line.split()
    print(words)
    break
    read_lengths.append(len(line[33:line.find('	',33)]))
    if(len(line[33:line.find('	',33)]) == 2013):
        print(line[33:line.find('	',33)])
    
#     print(line.strip())
    counter += 1
    sumOfItAll += len(line[33:line.find(' ',33)])
#     if(counter == 50):
#         break

print("average read length: ", sum(read_lengths)/len(read_lengths))
print("max read length: ", max(read_lengths))
print("min read length: ", min(read_lengths))

# print(read_lengths)
#     count += 1
#     print("Line{}: {}".format(count, line.strip()))

10001
0
['@SQ', 'SN:gi|9629357:1-9117', 'LN:9117']


ZeroDivisionError: division by zero

In [ ]:
sorted_list = sorted(read_lengths)
sorted_counted = Counter(sorted_list)

range_length = list(range(max(some_list))) # Get the largest value to get the range.
data_series = {}

for i in range_length:
    data_series[i] = 0 # Initialize series so that we have a template and we just have to fill in the values.

for key, value in sorted_counted.items():
    data_series[key] = value

data_series = pd.Series(data_series)
x_values = data_series.index

# you can customize the limits of the x-axis
# plt.xlim(0, max(some_list))
plt.bar(x_values, data_series.values)

plt.show() 